## Prepare progression data from userdata 

This notebook constructs the progression dataset for the model based on the data in the `userdata_filter.jl` file <br/>
the `userdata_filter.jl` file contains a row for each user. The first column is a dictionary containing the demographic info for that user, and the rest of the columns are lists that contain the data for each tick from the user profile (date, location, style, rating, etc). This notebook constructs a dataframe with a progression metric (`Rs`) and various summary statistics for each user's tick log. <br/>



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from scipy import stats
from random import randrange
from datetime import datetime
import seaborn as sns
import time


First load the userdata file and get rid of low tick counts in case this wasn't already done

In [2]:
#Path to the filtered userdata file
userdata_path = '.../userdata_filter.jl'

df = pd.read_json(userdata_path, lines=True)

min_months = 5 #there should be at least this many unique months when calculating the correlation coefficient
tick_cutoff=100 
df = df[df.Date.str.len() > tick_cutoff]
df=df.reset_index().reset_index()

Now define a function that converts rating data to numeric (score)

In [5]:
#List of YDS grades
grades_yds = ['5.0','5.1','5.2','5.3','5.4','5.5','5.6','5.7','5.7+','5.8-','5.8','5.8+','5.9-','5.9','5.9+',
              '5.10a',['5.10-','5.10a/b'],'5.10b',['5.10','5.10b/c'],'5.10c',['5.10+','5.10c/d'],'5.10d',
              '5.11a',['5.11-','5.11a/b'],'5.11b',['5.11','5.11b/c'],'5.11c',['5.11+','5.11c/d'],'5.11d',
              '5.12a',['5.12-','5.12a/b'],'5.12b',['5.12','5.12b/c'],'5.12c',['5.12+','5.12c/d'],'5.12d',
              '5.13a',['5.13-','5.13a/b'],'5.13b',['5.13','5.13b/c'],'5.13c',['5.13+','5.13c/d'],'5.13d',
              '5.14a',['5.14-','5.14a/b'],'5.14b',['5.14','5.14b/c'],'5.14c',['5.14+','5.14c/d'],'5.14d',
              '5.15a',['5.15-','5.15a/b'],'5.15b',['5.15','5.15b/c'],'5.15c',['5.15+','5.15c/d'],'5.15d']

#Grade index - half step for slash grades, fullstep for letter grades and number grades, 1/3 step for +- below 5.10
grade_index_yds = [0,1,2,3,4,5,6,7,7.33,7.66,8,8.33,8.66,9,9.5,
               10, 10.5, 11, 11.5, 12, 12.5, 13,
               14, 14.5, 15, 15.5, 16, 16.5, 17,
               18, 18.5, 19, 19.5, 20, 20.5, 21,
               22, 22.5, 23, 23.5, 24, 24.5, 25,
               26, 26.5, 27, 27.5, 28, 28.5, 29,
               30, 30.5, 31, 31.5, 32, 32.5, 33]

grade_index_yds = np.exp(np.array(grade_index_yds)/9)

#Repeat for Vgrades
grades_vscale = ['V-easy', 'V0-', 'V0', ['V0+', 'V1-'], 
                 'V1', ['V1+', 'V2-'], 'V2',['V2+','V3-'],
                 'V3', ['V3+', 'V4-'],'V4',['V4+','V5-'],
                 'V5',['V5+','V6-'],'V6',['V6+','V7-'],
                 'V7',['V7+','V8-'],'V8', ['V8+','V9-'],
                 'V9',['V9+','V10-'],'V10',['V10+','V11-'],
                 'V11',['V11+','V12-'],'V12', ['V12+','V13-'],
                 'V13',['V13+','V14-'],'V14',['V14+','V15-'],
                 'V15',['V15+','V16-'],'V16', ['V16+','V17-'],
                 'V17']

grade_index_vscale = [9, 9.5, 10, 11,
                      12 ,13 ,14, 15,
                      16, 17, 18, 19,
                      20, 21, 22, 23,
                      24, 25, 26, 27,
                      28, 29, 30, 31,
                      32, 33, 34, 35,
                      36, 37, 38, 39,
                      40, 41, 42, 43,
                      44]
grade_index_vscale = np.exp(np.array(grade_index_vscale)/9)


#Create a ref score that you can easily access - YDS
grades_yds_unpacked = []
ref_scores_yds = []
for i,grade in enumerate(grades_yds):
    if isinstance(grade,str):
        ref_scores_yds.append(grade_index_yds[i])
        grades_yds_unpacked.extend([grade])
    else:
        ref_scores_yds.extend(len(grade)*[grade_index_yds[i]])
        grades_yds_unpacked.extend(grade)                    
grades_yds = grades_yds_unpacked

#Create a ref score that you can easily access - Vscale
grades_vscale_unpacked = []
ref_scores_vscale = []
for i,grade in enumerate(grades_vscale):
    if isinstance(grade,str):
        ref_scores_vscale.append(grade_index_vscale[i])
        grades_vscale_unpacked.extend([grade])
    else:
        ref_scores_vscale.extend(len(grade)*[grade_index_vscale[i]])
        grades_vscale_unpacked.extend(grade)                    
grades_vscale = grades_vscale_unpacked

#Extend to make one huge array
grades_yds.extend(grades_vscale)
ref_scores_yds.extend(ref_scores_vscale)


def grade_to_score(input_grades):
    
    #Make input a list if it's a string
    if isinstance(input_grades, str):
        input_grades=[input_grades]
        
    #Convert VX-Y grades to VX+
    input_grades = [str(grade.split('-')[0]+'+') if bool(re.match(r"(V\d+-\d+)",grade)) 
                    else grade for grade in input_grades]

            
    #Convert grades that are not in the list to 5.0 (scrambles)
    input_grades = ['5.0' if x not in grades_yds else x for x in input_grades]
                  
    output_scores = [ref_scores_yds[grades_yds.index(x)] for x in input_grades] 
    return output_scores



Define a function that converts a row of the `userdata_filter.jl` file to a dataframe containing information about each tick on the profile (excludes demographic info)

In [6]:
def produce_ticks_filterdates(df, ind, mindate, maxdate):
    #Removes ticks with dates out of the range defined by mindate and max date
    #Takes as arg the entire user dataframe and the index of the user
    #outputs df_user with date converted to datetime.

    i=ind
    #Much faster method of doing things!
    df_user = pd.DataFrame.from_dict({'Date': pd.to_datetime(df.Date[i], errors='coerce', format='%b %d, %Y'),
                                      'RatingYDS': pd.Series(df.RatingYDS[i]).tolist(),
                                      'RatingVscale': pd.Series(df.RatingVscale[i]).tolist(),
                                      'RatingMisc': pd.Series(df.RatingMisc[i]).tolist(),
                                      'RouteType': pd.Series(df.RouteType[i]).tolist(),
                                      'Location': pd.Series(df.Location[i]).tolist(),
                                      'Style': pd.Series(df.Style[i]).tolist(),
                                      'Pitches': pd.Series(df.Pitches[i]).tolist(),
                                      'Notes': pd.Series(df.Notes[i]).tolist()})
    df_user = df_user.set_index('Date')
    df_user=df_user[df_user.index.to_series().between(mindate, maxdate)]
    df_user.reset_index(inplace=True) #convert date index back to a normal column....
    return df_user


Define a function that calculates the Spearmann correlation coefficient for climb ratings vs time as a measure of if the user is progressing in climb difficulty. <br/>

First we combine YDS and boulder ratings and convert that column to scores. <br/>
Then, since people can climb easy stuff on a given day (for fun, as a rest day, alpine climbing, etc), we take the highest graded route done in each month as a data point. <br/>
Then, we calculate the correlation coefficient with time 

In [12]:
#This function calculates the spearman coefficient for a user's tick df
def get_user_tickcorrelation(df_user, ploton): ## Given the tick dataframe for a single user
    #inputs is a dict with keys:factor for outlier removal and Ntake for taking highest ticks for each day
  
    #Filtering 
    df_user = df_user[df_user.Date.isna()==False] #Remove rows where date is NaN (out of bounds)
    df_user.RatingYDS.fillna(df_user.RatingVscale, inplace=True) #Fill empty YDS ratings with boulder ratings
    df_user.drop(columns=['RatingVscale'], inplace = True) #drop vscale column
    df_user.rename(columns={'RatingYDS':'Rating'},inplace=True) #rename ratingyds column
    df_user = df_user[df_user.Rating.isna()==False] #remove rows where there's no YDS rating
    scores = grade_to_score(df_user.Rating) #Calculate scores for the climbs
    df_user.drop('Rating',axis=1,inplace=True) #get rid of the rating column
    df_user['Rating']=scores #replace rating column with scores
    df_user = df_user[df_user.Rating > 0] #remove zero score values (scrambles/ice/missing values, etc)
    df_user.index=df_user.Date #set the date to be the index- necessary for grouping operation

    # Take the highest rated climb from each unique month in the user's ticks
    grouped = df_user.groupby(by=[df_user.index.year, df_user.index.month]) # Make date groups
    df_usercorr = pd.DataFrame.from_dict({'Rating':[]})
    df_usercorr.Rating = grouped.pipe(lambda x: x.Rating.max()) #Returns highest score in each group
    dates = [str(x[1])+'/'+str(x[0]) for x in df_usercorr.index] #date list (mont/year)
    df_usercorr['Date'] = pd.to_datetime(dates, errors='coerce', format='%m/%Y')
    
    #Calculate spearman coefficient
    x=pd.to_numeric(df_usercorr.Date).values*0.0000000000000001 #for plotting...
    y=df_usercorr.Rating.values
    rs = str(stats.spearmanr(x, y).correlation)
    
    if len(x) < min_months:
        rs = -100
    
    if ploton:
        x= x.reshape(-1, 1)
        reg = LinearRegression().fit(x, y)
        print('Spearman coeff = '+str(rs))
        sns.regplot(x=x,y=y)

    return float(rs)

Define a function to get demographics from a row of `userdata_filter.jl`

In [1]:
def get_user_demographics(UserInfo):
    #input: UserInfo (== df.UserInfo[i])
    
    if len(UserInfo['Gender'])==2: #Conversion failed in scraper
        gender=None
    else:
        gender=UserInfo['Gender']
    age=UserInfo['Age']
    accountage= ((pd.Timestamp.now()-pd.to_datetime(UserInfo['SignupDate'])).days)/365  #account age in years
    
    #Replace fake ages with None
    if age:
        if age<14 or age>80:
            age=None
    
    demovalues= {
        'Age':age,
        'Gender':gender,
        'AccountAge':accountage
    }
    
    return demovalues

Define a function to get a bunch of features from a user's tick log. These are: <br/>

`TickAge`: Date of most recent tick - date of first tick in years<br/>
`NumberTicks`: Total number of ticks<br/>
`NumberDays`: Number of unique days on which there is at least 1 tick<br/>
`FractionTrad`: Fraction of ticks that are trad routes<br/>
`FractionSport`: Fraction of ticks that are sport routes<br/>
`FractionBoulder`: Fraction of ticks that are boulders<br/>
`FractionAlpine`: Fraction of ticks that are alpine routes<br/>
`FractionMultipitch`: Fraction of ticks that are >1 pitch<br/>
`FractionDanger`: Fraction of ticks that have a danger rating (PG13, R)<br/>
`VarType`: Standard deviation in an integer encoding of route type<br/>
`VarLoc`: Number of unique areas ticks cover<br/>
`FractionStyleMarked`: Fraction of ticks where a style is marked<br/>
`FractionOnsight`: Fraction of ticks where the style is onsight<br/>
`FractionRedpoint`: Fraction of ticks where the style is redpoint<br/>
`FractionFell`: Fraction of ticks where the style is Lead/ Fell/Hung<br/>
`FractionSolo`: Fraction of ticks where the style is Solo<br/>
`FractionTR`: Fraction of ticks where the style is TR<br/>
`FractionDaysClimbing`: Fraction of all days in ticklog range that there are ticks<br/>

                 

In [9]:
def get_user_features(df_user):
#     INPUT: User tick dataframe. 
#     OUTPUT: dictionary with desired values.

    #Calculate ticklog_age
    date_series = df_user.Date[df_user.Date.isna()==False] #remove NaT
    tickage=((date_series.iloc[1]-date_series.iloc[-1]).days)/365 #tick age in years 
    
    #Number of days ticking
    ndays = df_user.Date.nunique()

    #Calculate the fraction of things
    nticks = len(df_user.index)
    ntrad = pd.Series((df_user.RouteType.str.contains('Trad')) & (df_user.RouteType.str.contains('Boulder')==False)).sum()/nticks
    nsport = pd.Series((df_user.RouteType.str.contains('Sport')) & (df_user.RouteType.str.contains('Boulder')==False)).sum()/nticks
    nboulder = pd.Series(df_user.RouteType.str.contains('Boulder')).sum()/nticks
    nalpine = pd.Series(df_user.RouteType.str.contains('Alpine')).sum()/nticks
    npitches = pd.Series(df_user.Pitches > 1).sum()/nticks
    ndanger = pd.Series(df_user.RatingMisc.str.contains('R')|df_user.RatingMisc.str.contains('PG13')).sum()/nticks
    
    #Diversity in climb types - replace values with number to indicate type
    mask_trad = (df_user.RouteType.astype(str).str.contains('Trad')) & (df_user.RouteType.astype(str).str.contains('Boulder')==False)
    df_user.loc[mask_trad, 'RouteType'] = 1
    mask_sport = (df_user.RouteType.astype(str).str.contains('Sport')) & (df_user.RouteType.astype(str).str.contains('Boulder')==False)
    df_user.loc[mask_sport, 'RouteType'] = 2
    mask_boulder = (df_user.RouteType.astype(str).str.contains('Boulder')==True)
    df_user.loc[mask_boulder, 'RouteType'] = 3
    mask_misc=(df_user.RouteType.astype(str).str.isdigit())==False
    df_user.loc[mask_misc, 'RouteType'] = 4
    VarType = df_user.RouteType.std()
    
    #Diversity in locations (#of different unique locations - at the sub state level)
    VarLoc = df_user.Location.str.split(',').apply(lambda x: x[0] if x else 0).nunique()
    
    #Fraction of ticks that have a style marked 
    FractionStyleMarked = sum(df_user.Style.isna()==False)/len(df_user.index) #fraction of style that are filled
    Nmarked = sum(df_user.Style.isna()==False)
    styles = df_user.Style[df_user.Style.isna()==False]
    if Nmarked > 0:
        FractionOnsight = (sum(styles.str.contains('Onsight'))+sum(styles.str.contains('Flash')))/Nmarked
        FractionRedpoint = (sum(styles.str.contains('Send'))+sum(styles.str.contains('Redpoint')))/Nmarked
        FractionFell = (sum(styles.str.contains('Fell'))+sum(styles.str.contains('Attempt')))/Nmarked
        FractionSolo=(sum(styles.str.contains('Solo')))/Nmarked
        FractionTR = (sum(styles.str.contains('TR'))+sum(styles.str.contains('Follow')))/Nmarked
    else:
        FractionOnsight=None
        FractionRedpoint=None
        FractionFell=None
        FractionSolo=None
        FractionTR=None

    #Fraction of days spent climbing
    FractionDaysClimbing = df_user.Date.nunique()/(max(df_user.Date)-min(df_user.Date)).days
    

    
    output_dict={'TickAge':tickage,
                 'NumberTicks':nticks,
                 'NumberDays': ndays,
                 'FractionTrad':ntrad,
                 'FractionSport':nsport,
                 'FractionBoulder':nboulder,
                 'FractionAlpine':nalpine,
                 'FractionMultipitch':npitches,
                 'FractionDanger':ndanger,
                 'VarType': VarType,
                 'VarLoc': VarLoc,
                 'FractionStyleMarked':FractionStyleMarked,
                 'FractionOnsight':FractionOnsight,
                 'FractionRedpoint':FractionRedpoint,
                 'FractionFell':FractionFell,
                 'FractionSolo': FractionSolo,
                 'FractionTR': FractionTR,
                 'FractionDaysClimbing': FractionDaysClimbing
                }
    return output_dict

## Build the dataframe 

In [20]:
#Initialize 

#Date range to filter ticks
mindate = '2000-01-01'
maxdate = '2023-01-01' 

tick_cutoff=50 #minimum number of ticks
days_cutoff=15 #minimum number of unique days

#Initialize empty dataframe
initial = len(df.index)*[0]
userdata = pd.DataFrame.from_dict({
    'Rs': initial,
    'Age': initial,
    'Gender': initial,
    'AccountAge':initial,
    'TickAge':initial,
    'NumberTicks':initial,
    'NumberDays':initial,
    'FractionTrad':initial,
    'FractionSport':initial,
    'FractionBoulder':initial,
    'FractionAlpine':initial,
    'FractionMultipitch':initial,
    'FractionDanger':initial,
    'VarType': initial,
    'VarLoc': initial,
    'FractionStyleMarked':initial,
    'FractionOnsight':initial,
    'FractionRedpoint':initial,
    'FractionFell':initial,
    'FractionSolo': initial,
    'FractionTR': initial,
    'FractionDaysClimbing': initial
})


In [21]:
#######################################
#Build the dataframe iteratively
#######################################

for i in range(len(df.index)): #for each user
    
    lb = time.time()
    #Get user ticks and filter out of bounds dates
    df_user = produce_ticks_filterdates(df, i, mindate, maxdate)
    
    #Calculate the number of unique days tickin
    
    #Add to main dataframe ~if~ there are > tick_cutoff ticks remaining (after date filtering ...)
    if df_user.Date.nunique() >= days_cutoff:

        #Calculate corr coeff.
        Rs = get_user_tickcorrelation(df_user, False) #false for not plotting
        
        #Get demographic features as dictionary
        demo_features = get_user_demographics(df.UserInfo[i])
        
        #Get all features as a dictionary
        features = get_user_features(df_user)

    else: #Add none for entire row to avoid issues.filter these out later
        Rs =  None
        demo_features ={'Age':None,
                        'Gender':None,
                        'AccountAge':None
        }
        features={'TickAge':None,
                 'NumberTicks':None,
                 'NumberDays':None,
                 'FractionTrad':None,
                 'FractionSport':None,
                 'FractionBoulder':None,
                 'FractionAlpine':None,
                 'FractionMultipitch':None,
                 'FractionDanger':None,
                 'VarType': None,
                 'VarLoc': None,
                 'FractionStyleMarked':None,
                 'FractionOnsight':None,
                 'FractionRedpoint':None,
                 'FractionFell':None,
                 'FractionSolo': None,
                 'FractionTR': None,
                 'FractionDaysClimbing': None
                }
        
        
    #Assign values
    userdata.loc[i,'Rs']=Rs
    userdata.loc[i,'Age']=demo_features['Age']
    userdata.loc[i,'Gender']=demo_features['Gender']
    userdata.loc[i,'AccountAge']=demo_features['AccountAge']
    userdata.loc[i,'TickAge']=features['TickAge']
    userdata.loc[i,'NumberTicks']=features['NumberTicks']
    userdata.loc[i,'NumberDays']=features['NumberDays']
    userdata.loc[i,'FractionTrad']=features['FractionTrad']
    userdata.loc[i,'FractionSport']=features['FractionSport']
    userdata.loc[i,'FractionBoulder']=features['FractionBoulder']
    userdata.loc[i,'FractionAlpine']=features['FractionAlpine']
    userdata.loc[i,'FractionMultipitch']=features['FractionMultipitch']
    userdata.loc[i,'FractionDanger']=features['FractionDanger']
    userdata.loc[i,'VarType']=features['VarType']
    userdata.loc[i,'VarLoc']=features['VarLoc']
    userdata.loc[i,'FractionStyleMarked']=features['FractionStyleMarked']
    userdata.loc[i,'FractionOnsight']=features['FractionOnsight']
    userdata.loc[i,'FractionRedpoint']=features['FractionRedpoint']
    userdata.loc[i,'FractionFell']=features['FractionFell']
    userdata.loc[i,'FractionSolo']=features['FractionSolo']
    userdata.loc[i,'FractionTR']=features['FractionTR']
    userdata.loc[i,'FractionDaysClimbing']=features['FractionDaysClimbing']
    
    #Output time and index value
    ub = time.time()
    print('i='+str(i)+', Time to convert = '+str(ub-lb))
    
    

i=0, Time to convert = 0.04510641098022461
i=1, Time to convert = 0.025039196014404297
i=2, Time to convert = 0.025049209594726562
i=3, Time to convert = 0.025036096572875977
i=4, Time to convert = 0.025065898895263672
i=5, Time to convert = 0.025034666061401367
i=6, Time to convert = 0.030047893524169922
i=7, Time to convert = 0.02505016326904297
i=8, Time to convert = 0.023564577102661133
i=9, Time to convert = 0.03008127212524414
i=10, Time to convert = 0.030084848403930664
i=11, Time to convert = 0.020055294036865234
i=12, Time to convert = 0.030083894729614258
i=13, Time to convert = 0.030091047286987305
i=14, Time to convert = 0.028069019317626953
i=15, Time to convert = 0.027576923370361328
i=16, Time to convert = 0.022564411163330078
i=17, Time to convert = 0.0325927734375
i=18, Time to convert = 0.025072813034057617
i=19, Time to convert = 0.02505779266357422
i=20, Time to convert = 0.027574539184570312
i=21, Time to convert = 0.0276031494140625
i=22, Time to convert = 0.02955

i=190, Time to convert = 0.030061006546020508
i=191, Time to convert = 0.0245358943939209
i=192, Time to convert = 0.02505040168762207
i=193, Time to convert = 0.030057191848754883
i=194, Time to convert = 0.025035381317138672
i=195, Time to convert = 0.010018348693847656
i=196, Time to convert = 0.02504563331604004
i=197, Time to convert = 0.02503228187561035
i=198, Time to convert = 0.025084972381591797
i=199, Time to convert = 0.030086994171142578
i=200, Time to convert = 0.030061721801757812
i=201, Time to convert = 0.024532794952392578
i=202, Time to convert = 0.02505326271057129
i=203, Time to convert = 0.025043487548828125
i=204, Time to convert = 0.025043964385986328
i=205, Time to convert = 0.02504134178161621
i=206, Time to convert = 0.025046110153198242
i=207, Time to convert = 0.02706003189086914
i=208, Time to convert = 0.02805495262145996
i=209, Time to convert = 0.025048494338989258
i=210, Time to convert = 0.03004598617553711
i=211, Time to convert = 0.02955484390258789

i=377, Time to convert = 0.05512714385986328
i=378, Time to convert = 0.025039196014404297
i=379, Time to convert = 0.02505183219909668
i=380, Time to convert = 0.025038957595825195
i=381, Time to convert = 0.029546022415161133
i=382, Time to convert = 0.02504587173461914
i=383, Time to convert = 0.0250399112701416
i=384, Time to convert = 0.025048017501831055
i=385, Time to convert = 0.030052900314331055
i=386, Time to convert = 0.025036334991455078
i=387, Time to convert = 0.010022640228271484
i=388, Time to convert = 0.025049448013305664
i=389, Time to convert = 0.02003335952758789
i=390, Time to convert = 0.03505873680114746
i=391, Time to convert = 0.0250546932220459
i=392, Time to convert = 0.025040388107299805
i=393, Time to convert = 0.03456306457519531
i=394, Time to convert = 0.0300748348236084
i=395, Time to convert = 0.030055761337280273
i=396, Time to convert = 0.02504134178161621
i=397, Time to convert = 0.02504730224609375
i=398, Time to convert = 0.02503514289855957
i=3

i=560, Time to convert = 0.027579545974731445
i=561, Time to convert = 0.024547100067138672
i=562, Time to convert = 0.025083303451538086
i=563, Time to convert = 0.02455759048461914
i=564, Time to convert = 0.026067733764648438
i=565, Time to convert = 0.024038314819335938
i=566, Time to convert = 0.03507232666015625
i=567, Time to convert = 0.020030736923217773
i=568, Time to convert = 0.035097599029541016
i=569, Time to convert = 0.025063276290893555
i=570, Time to convert = 0.025075912475585938
i=571, Time to convert = 0.029573440551757812
i=572, Time to convert = 0.030104875564575195
i=573, Time to convert = 0.02357935905456543
i=574, Time to convert = 0.02809739112854004
i=575, Time to convert = 0.032103776931762695
i=576, Time to convert = 0.00802922248840332
i=577, Time to convert = 0.028071880340576172
i=578, Time to convert = 0.035068511962890625
i=579, Time to convert = 0.025072336196899414
i=580, Time to convert = 0.032073020935058594
i=581, Time to convert = 0.028064727783

i=742, Time to convert = 0.03008127212524414
i=743, Time to convert = 0.030096769332885742
i=744, Time to convert = 0.030072689056396484
i=745, Time to convert = 0.04009389877319336
i=746, Time to convert = 0.025073528289794922
i=747, Time to convert = 0.0296022891998291
i=748, Time to convert = 0.03507113456726074
i=749, Time to convert = 0.030057668685913086
i=750, Time to convert = 0.030060291290283203
i=751, Time to convert = 0.02505636215209961
i=752, Time to convert = 0.0300600528717041
i=753, Time to convert = 0.03505277633666992
i=754, Time to convert = 0.025046110153198242
i=755, Time to convert = 0.03008556365966797
i=756, Time to convert = 0.03976726531982422
i=757, Time to convert = 0.03009653091430664
i=758, Time to convert = 0.030047893524169922
i=759, Time to convert = 0.040091753005981445
i=760, Time to convert = 0.049722909927368164
i=761, Time to convert = 0.030219078063964844
i=762, Time to convert = 0.03507828712463379
i=763, Time to convert = 0.028435468673706055
i

...

i=27789, Time to convert = 0.010000944137573242
i=27790, Time to convert = 0.026975631713867188
i=27791, Time to convert = 0.026975154876708984
i=27792, Time to convert = 0.008992433547973633
i=27793, Time to convert = 0.02697587013244629
i=27794, Time to convert = 0.010000228881835938
i=27795, Time to convert = 0.023979663848876953
i=27796, Time to convert = 0.026975154876708984
i=27797, Time to convert = 0.02697587013244629
i=27798, Time to convert = 0.02399730682373047
i=27799, Time to convert = 0.028017520904541016
i=27800, Time to convert = 0.02498459815979004
i=27801, Time to convert = 0.023980379104614258
i=27802, Time to convert = 0.02697467803955078
i=27803, Time to convert = 0.008990764617919922
i=27804, Time to convert = 0.027981996536254883
i=27805, Time to convert = 0.02397894859313965
i=27806, Time to convert = 0.026976346969604492
i=27807, Time to convert = 0.011989593505859375
i=27808, Time to convert = 0.027991294860839844
i=27809, Time to convert = 0.02696919441223144

i=27970, Time to convert = 0.028035640716552734
i=27971, Time to convert = 0.010008811950683594
i=27972, Time to convert = 0.025012731552124023
i=27973, Time to convert = 0.027039527893066406
i=27974, Time to convert = 0.023006677627563477
i=27975, Time to convert = 0.02704024314880371
i=27976, Time to convert = 0.026005983352661133
i=27977, Time to convert = 0.010010957717895508
i=27978, Time to convert = 0.027029752731323242
i=27979, Time to convert = 0.007997512817382812
i=27980, Time to convert = 0.03302717208862305
i=27981, Time to convert = 0.03097820281982422
i=27982, Time to convert = 0.026975631713867188
i=27983, Time to convert = 0.026976585388183594
i=27984, Time to convert = 0.023995161056518555
i=27985, Time to convert = 0.02703094482421875
i=27986, Time to convert = 0.026009082794189453
i=27987, Time to convert = 0.024077892303466797
i=27988, Time to convert = 0.02696537971496582
i=27989, Time to convert = 0.024987459182739258
i=27990, Time to convert = 0.0269765853881835

c:\users\doug\.julia\conda\3\lib\site-packages\scipy\stats\_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


i=28005, Time to convert = 0.02799510955810547
i=28006, Time to convert = 0.026964426040649414
i=28007, Time to convert = 0.026976585388183594
i=28008, Time to convert = 0.026974916458129883
i=28009, Time to convert = 0.024987220764160156
i=28010, Time to convert = 0.026974201202392578
i=28011, Time to convert = 0.023978471755981445
i=28012, Time to convert = 0.02697587013244629
i=28013, Time to convert = 0.02800750732421875
i=28014, Time to convert = 0.028006553649902344
i=28015, Time to convert = 0.026054859161376953
i=28016, Time to convert = 0.02801060676574707
i=28017, Time to convert = 0.010011434555053711
i=28018, Time to convert = 0.02505040168762207
i=28019, Time to convert = 0.010009765625
i=28020, Time to convert = 0.030030488967895508
i=28021, Time to convert = 0.027048349380493164
i=28022, Time to convert = 0.010025262832641602
i=28023, Time to convert = 0.02801799774169922
i=28024, Time to convert = 0.026963233947753906
i=28025, Time to convert = 0.03297066688537598
i=280

i=28186, Time to convert = 0.03508734703063965
i=28187, Time to convert = 0.010029077529907227
i=28188, Time to convert = 0.030058860778808594
i=28189, Time to convert = 0.010023117065429688
i=28190, Time to convert = 0.010022163391113281
i=28191, Time to convert = 0.03510260581970215
i=28192, Time to convert = 0.03055548667907715
i=28193, Time to convert = 0.009537696838378906
i=28194, Time to convert = 0.025040388107299805
i=28195, Time to convert = 0.010027170181274414
i=28196, Time to convert = 0.02959442138671875
i=28197, Time to convert = 0.03005814552307129
i=28198, Time to convert = 0.030053138732910156
i=28199, Time to convert = 0.03008866310119629
i=28200, Time to convert = 0.035111188888549805
i=28201, Time to convert = 0.024541616439819336
i=28202, Time to convert = 0.035123586654663086
i=28203, Time to convert = 0.03507494926452637
i=28204, Time to convert = 0.025050878524780273
i=28205, Time to convert = 0.010022401809692383
i=28206, Time to convert = 0.010022640228271484

In [27]:
#Save to a file after removing NaN rows:

userdata = userdata[userdata.Rs.isna()==False]# Replace ridiculous ages with NaN
userdata = userdata[userdata.Rs > -1] #Remove rows where min_months criteria wasn't met

filepath = '.../userdata_formatted.json' #Path to file
userdata.to_json(filepath)
